UMLS的数据分析

In [ ]:
import jsonlines
import pandas as pd

instructions = list(jsonlines.open("../instruction_datasets/instruction_dataall.jsonl"))
df = pd.read_csv("./UMLS/umls_kg_filter.csv")
# mrdef = pd.read_csv("./UMLS/MRDEF_name.csv",error_bad_lines=False)


In [3]:
source_counts = df['source'].value_counts()
target_counts = df['target'].value_counts()
edge_counts = df['edge'].value_counts()
import plotly.express as px
px.pie(edge_counts, values=edge_counts.values, names=edge_counts.index, title='edge distribution')

In [5]:
from tqdm.auto import tqdm
import spacy
nlp = spacy.load("en_core_web_sm")
import random

i = random.choice(instructions)
prompt = i['input'].strip()+" "+i['output'].strip()
doc = nlp(prompt)
from spacy import displacy
displacy.render(doc, style="ent")

In [1]:
import pickle

dst = pickle.load(open('data/EntityDataset_1000.pkl','rb'))
tok = 

In [3]:
def show(d):
    input_ids, attention_mask, labels, hard_position_type_ids = d[0],d[1],d[2],d[3]
    tokens = 
show(dst[0])